In [2]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

class DiscreteSignal:
    color_cycle = itertools.cycle(['b', 'g', 'r', 'c', 'm', 'y', 'k'])
    def __init__(self, INF):
        self.INF = INF
        self.values = np.zeros(2 * INF + 1)
    def set_value_at_time(self, time, value):
       self.values[time + self.INF] = value    
    
    def get_value_at_time(self, time):
       return self.values[time + self.INF] 
    def shift_signal(self, shift):
        if abs(shift) > self.INF*2+1:
            self.values = np.zeros(2*self.INF+1)
            return
        if shift > 0:
            self.values = np.concatenate((np.zeros(shift), self.values[0:2*self.INF+1-shift]))
        elif shift < 0:
            self.values = np.concatenate((self.values[-shift:], np.zeros(-shift)))
    
    def add(self,other):
        signal2=DiscreteSignal(self.INF)
        signal2.values=self.values+other.values
        return signal2
    def multiply(self,other):
        signal2=DiscreteSignal(self.INF)
        signal2.values=self.values*other.values
        return signal2  
    def multiply_const_factor(self, scaler):
        signal2=DiscreteSignal(self.INF)
        signal2.values=self.values*scaler
        return signal2
    def plot(self, label='Signal'):
        time_points = np.arange(-self.INF, self.INF + 1, 1)
        color = next(DiscreteSignal.color_cycle)
        plt.stem(time_points, self.values, basefmt=" ", label=label, linefmt=color, markerfmt=color+'o')
        plt.xlabel('n (Time Index)')
        plt.ylabel('x[n]')
        plt.legend()
        plt.grid(True)
        # plt.show()
        # plt.figure()
        
    
        


NameError: name 'DiscreteSignal' is not defined

In [ ]:
# Main function to generate subplots and save figures

def main():
    # Set up the impulse response and input signal
    INF = 5
    impulse_response = DiscreteSignal(INF)
    impulse_response.set_value_at_time(0, 1)
    impulse_response.set_value_at_time(1, 1)
    impulse_response.set_value_at_time(2, 1)
    
    input_signal = DiscreteSignal(INF)
    input_signal.set_value_at_time(0, 0.5)
    input_signal.set_value_at_time(1, 2)

    # Initialize the LTI system
    lti_system = LTI_Discrete(impulse_response)
    
    # Plot the input signal
    plt.figure()
    input_signal.plot("")
    plt.title("Input Discrete Signal, INF=5")
    plt.savefig("discrete_plots/input_signal.png")


# Run the main function
main()
